<a href="https://colab.research.google.com/github/agrigoridou/online_shoppers_intention/blob/main/online_shoppers_intention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports & Basic Setup

In [ ]:
# Βασικά πακέτα ανάλυσης δεδομένων
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# Warnings off
import warnings
warnings.filterwarnings("ignore")


Load Dataset

In [ ]:
# Φόρτωση dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00468/online_shoppers_intention.csv"
df = pd.read_csv(url)

df.head()


Basic Info + Missing Values

In [ ]:
df.info()
df.isnull().sum()


Basic Statistics

In [ ]:
df.describe(include='all')


Label Encoding (μετατροπή κατηγορικών σε αριθμούς)

In [ ]:
# Αντιγραφή dataframe
data = df.copy()

# Label Encoding για Month & VisitorType
le = LabelEncoder()
cat_cols = ["Month", "VisitorType"]

for col in cat_cols:
    data[col] = le.fit_transform(data[col])

# Boolean σε 0/1
data["Weekend"] = data["Weekend"].astype(int)
data["Revenue"] = data["Revenue"].astype(int)

data.head()


Correlation Heatmap

In [ ]:
plt.figure(figsize=(16,12))
sns.heatmap(data.corr(), cmap="coolwarm")
plt.title("Correlation Heatmap")
plt.show()


Pairplot / Exploratory Visualizations

In [ ]:
sns.pairplot(data[['Administrative', 'Informational', 'ProductRelated', 'BounceRates', 'ExitRates', 'Revenue']], hue='Revenue')
plt.show()


Define X and y

In [ ]:
X = data.drop("Revenue", axis=1)
y = data["Revenue"]


Train/Test Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


Feature Scaling

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


Logistic Regression Model

In [ ]:
log_model = LogisticRegression(max_iter=1000)
log_model.fit(X_train_scaled, y_train)

y_pred_log = log_model.predict(X_test_scaled)

print("Accuracy:", accuracy_score(y_test, y_pred_log))
print("\nClassification Report:\n", classification_report(y_test, y_pred_log))
sns.heatmap(confusion_matrix(y_test, y_pred_log), annot=True, fmt="d")
plt.title("Confusion Matrix - Logistic Regression")
plt.show()


Random Forest Model

In [ ]:
rf = RandomForestClassifier(n_estimators=200, random_state=42)
rf.fit(X_train, y_train)

y_pred_rf = rf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred_rf))
print("\nClassification Report:\n", classification_report(y_test, y_pred_rf))
sns.heatmap(confusion_matrix(y_test, y_pred_rf), annot=True, fmt="d")
plt.title("Confusion Matrix - Random Forest")
plt.show()


Feature Importance (Random Forest)

In [ ]:
importances = pd.Series(rf.feature_importances_, index=X.columns)

plt.figure(figsize=(12,6))
importances.sort_values().plot(kind="barh")
plt.title("Feature Importances")
plt.show()


Save Model for Deployment

In [ ]:
import joblib

joblib.dump(rf, "shopping_rf_model.pkl")
joblib.dump(scaler, "scaler.pkl")


Predict on Custom User Input (Interactive Example)

In [ ]:
sample = X_test.iloc[0:1]
print("Sample Input:")
print(sample)

sample_scaled = scaler.transform(sample)

print("\nRandom Forest Prediction:", rf.predict(sample)[0])
print("Logistic Regression Prediction:", log_model.predict(sample_scaled)[0])
